In [1]:
import numpy as np
import pandas as pd
import h5py
import time
from sklearn.utils.extmath import randomized_svd
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.manifold import TSNE
%matplotlib inline

In [2]:
# Import data
df = pd.read_csv("raw_data/train_triplets.txt",delimiter="\t",header=None,names=["User","Song","Count"])

In [3]:
# Create sparse matrix
users = sorted(df.User.unique())
songs = sorted(df.Song.unique())

data = df['Count'].tolist()
row = df.Song.astype('category', categories=songs).cat.codes
col = df.User.astype('category', categories=users).cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(songs), len(users)))

In [4]:
# Weighted matrix factorization taken from Sander Dieleman
def iter_rows(S):
    """
    Helper function to iterate quickly over the data and indices of the
    rows of the S matrix. A naive implementation using indexing
    on S is much, much slower.
    """
    for i in range(S.shape[0]):
        lo, hi = S.indptr[i], S.indptr[i + 1]
        yield i, S.data[lo:hi], S.indices[lo:hi]

def recompute_factors(Y, S, lambda_reg, dtype='float32'):
    """
    recompute matrix X from Y.
    X = recompute_factors(Y, S, lambda_reg)
    This can also be used for the reverse operation as follows:
    Y = recompute_factors(X, ST, lambda_reg)
    
    The comments are in terms of X being the users and Y being the items.
    """
    m = S.shape[0] # m = number of users
    f = Y.shape[1] # f = number of factors
    YTY = np.dot(Y.T, Y) # precompute this
    YTYpI = YTY + lambda_reg * np.eye(f)
    X_new = np.zeros((m, f), dtype=dtype)

    for k, s_u, i_u in iter_rows(S):
        Y_u = Y[i_u] # exploit sparsity
        A = np.dot(s_u + 1, Y_u)
        YTSY = np.dot(Y_u.T, (Y_u * s_u.reshape(-1, 1)))
        B = YTSY + YTYpI

        # Binv = np.linalg.inv(B)
        # X_new[k] = np.dot(A, Binv) 
        X_new[k] = np.linalg.solve(B.T, A.T).T # doesn't seem to make much of a difference in terms of speed, but w/e

    return X_new

def factorize(S, num_factors, lambda_reg=1e-5, num_iterations=20, init_std=0.01, verbose=False, dtype='float32', recompute_factors=recompute_factors, *args, **kwargs):
    """
    factorize a given sparse matrix using the Weighted Matrix Factorization algorithm by
    Hu, Koren and Volinsky.

    S: 'surplus' confidence matrix, i.e. C - I where C is the matrix with confidence weights.
        S is sparse while C is not (and the sparsity pattern of S is the same as that of
        the preference matrix, so it doesn't need to be specified separately).

    num_factors: the number of factors.

    lambda_reg: the value of the regularization constant.

    num_iterations: the number of iterations to run the algorithm for. Each iteration consists
        of two steps, one to recompute U given V, and one to recompute V given U.

    init_std: the standard deviation of the Gaussian with which V is initialized.

    verbose: print a bunch of stuff during training, including timing information.

    dtype: the dtype of the resulting factor matrices. Using single precision is recommended,
        it speeds things up a bit.

    recompute_factors: helper function that implements the inner loop.

    returns:
        U, V: factor matrices. If bias=True, the last columns of the matrices contain the biases.
    """
    num_users, num_items = S.shape

    if verbose:
        print("precompute transpose")
        start_time = time.time()

    ST = S.T.tocsr()

    if verbose:
        print("  took %.3f seconds" % (time.time() - start_time))
        print("run ALS algorithm")
        start_time = time.time()

    U = None # no need to initialize U, it will be overwritten anyway
    V = np.random.randn(num_items, num_factors).astype(dtype) * init_std

    for i in range(num_iterations):
        if verbose:
            print("  iteration %d" % i)
            print("    recompute user factors U")

        U = recompute_factors(V, S, lambda_reg, dtype, *args, **kwargs)

        if verbose:
            print("    time since start: %.3f seconds" % (time.time() - start_time))
            print("    recompute item factors V")

        V = recompute_factors(U, ST, lambda_reg, dtype, *args, **kwargs)

        if verbose:
            print("    time since start: %.3f seconds" % (time.time() - start_time))

    return U, V

In [5]:
# WMF
U,V = factorize(sparse_matrix,10,verbose=True)

precompute transpose
  took 5.856 seconds
run ALS algorithm
  iteration 0
    recompute user factors U
    time since start: 25.857 seconds
    recompute item factors V
    time since start: 80.377 seconds
  iteration 1
    recompute user factors U
    time since start: 105.325 seconds
    recompute item factors V
    time since start: 159.654 seconds
  iteration 2
    recompute user factors U
    time since start: 184.634 seconds
    recompute item factors V
    time since start: 240.483 seconds
  iteration 3
    recompute user factors U
    time since start: 265.426 seconds
    recompute item factors V
    time since start: 321.089 seconds
  iteration 4
    recompute user factors U
    time since start: 346.133 seconds
    recompute item factors V
    time since start: 402.125 seconds
  iteration 5
    recompute user factors U
    time since start: 427.002 seconds
    recompute item factors V
    time since start: 481.308 seconds
  iteration 6
    recompute user factors U
    time si

In [10]:
# Store data in hdf5 format
f = h5py.File("data/Y.hdf5", "w")
f.create_dataset("Y", data=U)
ascii_songs = [n.encode("ascii", "ignore") for n in songs]
f.create_dataset('songs', (len(ascii_songs),1), 'S18', ascii_songs)
f.close()